In [ ]:
import numpy as np
import random
import csv
from Bio import SeqIO
from tensorflow.keras.utils import plot_model, model_to_dot
import tensorflow as tf
from tensorflow.keras.metrics import AUC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, LSTM, Dense, TimeDistributed, Bidirectional
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
def f1_score(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
def negative_predictive_value(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    total_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (total_negatives + K.epsilon())

def one_hot_decode(encoded_seq):
    mapping = {tuple([1, 0, 0, 0]): 'A',
               tuple([0, 1, 0, 0]): 'C',
               tuple([0, 0, 1, 0]): 'G',
               tuple([0, 0, 0, 1]): 'T',
               tuple([0, 0, 0, 0]): 'x'}
    return ''.join(mapping[tuple(nt)] for nt in encoded_seq)

def one_hot_encode(sequence):
    mapping = {'A': [1, 0, 0, 0], 'C': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'T': [0, 0, 0, 1]}
    return np.array([mapping[nt] for nt in sequence])


In [ ]:

# učitavanje podataka za treniranje

gff_file = '/path'
fna_file = '/path'

genes = []
with open(gff_file, 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        if row and not row[0].startswith('#') and row[2] == 'gene':
            start = int(row[3]) - 1
            end = int(row[4])
            genes.append(( start, end))

genome_array = []
label_array = []
for record in SeqIO.parse(fna_file, 'fasta'):
    genome_array = one_hot_encode(str(record.seq))
    label_array = np.zeros(len(record.seq))
    for  start, end in genes:
        label_array[start:end] = 1
block_size = 500
step_size = 250  # Step size for the sliding window
genome_blocks = [genome_array[i:i+block_size] for i in range(0, len(genome_array) - block_size + 1, step_size)]
label_blocks = [label_array[i:i+block_size] for i in range(0, len(label_array) - block_size + 1, step_size)]


data = list(zip(X, y))
random.shuffle(data)
X, y = zip(*data)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test,y_test= X,y
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)[:,:,None]
y_test = np.array(y_test)[:,:,None]



4494


In [ ]:
model = Sequential([
    Conv1D(filters=32, kernel_size=9, activation='relu', padding='same', input_shape=(500, 4)),
    Dropout(0.2),
    Conv1D(filters=64, kernel_size=9, activation='relu', padding='same'),
    Dropout(0.2),
    Bidirectional(LSTM(256, return_sequences=True)),
    Bidirectional(LSTM(256, return_sequences=True)),
    TimeDistributed(Dense(256, activation='relu')),
    TimeDistributed(Dense(128, activation='relu')),
    TimeDistributed(Dense(1, activation='sigmoid'))
])

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy',negative_predictive_value, Precision(name='precision'), Recall(name='recall'), f1_score])

model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2)
#model.save('')
loss, accuracy,Npv,precision,recall,f1_score = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy:.2f}')
print(loss)


In [ ]:

model=load_model("/path")

In [ ]:
#učitavanje podataka za predviđanje 
gff_file = '/path'
fna_file = '/path'

genes = []
with open(gff_file, 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        if row and not row[0].startswith('#') and row[2] == 'gene':
            start = int(row[3]) - 1
            end = int(row[4])
            genes.append(( start, end))

genome_array = []
label_array = []
for record in SeqIO.parse(fna_file, 'fasta'):
    genome_array = one_hot_encode(str(record.seq))
    label_array = np.zeros(len(record.seq))
    for  start, end in genes:
        label_array[start:end] = 1
block_size = 500
genome_blocks = [genome_array[i:i+block_size] for i in range(0, len(genome_array), block_size)]
label_blocks = [label_array[i:i+block_size] for i in range(0, len(label_array), block_size)]

genome_blocks = [block for block in genome_blocks if len(block) == block_size]
label_blocks = [block for block in label_blocks if len(block) == block_size]

X = np.array(genome_blocks)
y = np.array(label_blocks)

X_test,y_test= X,y
X_test = np.array(X_test)
y_test = np.array(y_test)[:,:,None]



In [ ]:

predictions = model.predict(X_test)
binary_predictions = np.where(predictions > 0.45, 1, 0)

In [ ]:
#ispis sekvence na kojoj je označeno što je kodirajući a sto nekodirajući dio sekvence
predseq=[]
for j in range(0,len(binary_predictions)):
    test1=binary_predictions[j]
    for i in range(0,len(test1)):
        #print(test1[i])
        if test1[i][0]==1:
            predseq.append(X_test[j][i])
        else:
            predseq.append(tuple([0, 0, 0, 0]))
        
print(len(predseq))
sequence = one_hot_decode(predseq)
#print(sequence)
    

In [ ]:
# brojanje gena i ispis dobivenih gena 
def count_coding_sequences(dna_seq):
    parts = [s for s in dna_seq.split('X') if len(s)>50]
    for i in range(10):  
        print(parts[i],"\n")
    count = len(parts)

    return count
print(count_coding_sequences(sequence))

In [ ]:
# isti proces kao u čelijama gore samo za stvarne gene
predseq=[]
for j in range(0,len(y_test)):
    test2=y_test[j]
    for i in range(0,len(test2)):
        if test2[i]==1:
            predseq.append(X_test[j][i])
        else:
            predseq.append(tuple([0, 0, 0, 0]))
        
print(len(predseq))
sequence = one_hot_decode(predseq)
#print(sequence)

In [ ]:
def count_coding_sequences(dna_seq):
    parts = [s for s in dna_seq.split('X') if s]
    for i in range(10): 
        print(parts[i],"\n")

    count = len(parts)

    return count
print(count_coding_sequences(sequence))

In [ ]:
test_fna_file = 'path'
test_gff_file = 'path'

test_genes = []
with open(test_gff_file, 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        if row and not row[0].startswith('#') and row[2] == 'gene':
            start = int(row[3]) - 1
            end = int(row[4])
            strand = 1 if row[6] == '+' else -1
            test_genes.append((start, end))

test_genome_array = []
test_label_array = []
for record in SeqIO.parse(test_fna_file, 'fasta'):
    test_genome_array = one_hot_encode(str(record.seq))
    test_label_array = np.zeros(len(record.seq))
    for start, end in test_genes:
        test_label_array[start:end] = 1

block_size = 500
step_size = 250

test_genome_blocks = [test_genome_array[i:i+block_size] for i in range(0, len(test_genome_array) - block_size + 1, step_size)]
test_label_blocks = [test_label_array[i:i+block_size] for i in range(0, len(test_label_array) - block_size + 1, step_size)]

test_genome_blocks = [block for block in test_genome_blocks if len(block) == block_size]
test_label_blocks = [block for block in test_label_blocks if len(block) == block_size]

X_test = np.array(test_genome_blocks)
y_test = np.array(test_label_blocks)

y_test = y_test[:,:,None]
predictions = model.predict(X_test)
final_prediction = np.zeros(len(test_genome_array))

for i, block in enumerate(predictions):
    start_index = i * step_size
    end_index = start_index + block_size
    final_prediction[start_index:end_index] += block.flatten()



In [ ]:
#ispis predikcija
binary_predictions = np.where(final_prediction > 1, 1, 0)
for i in range(len(binary_predictions)):
    print(binary_predictions[i],end = "")
    #print(int(y_test.flatten()[i]))

In [ ]:
#ispis predikcije u obliku sekvence genoma
predseq=[]
for j in range(0,len(binary_predictions[:6000]),block_size):
    test1=binary_predictions[j:j+block_size]
    for i in range(0,len(test1)):
        if test1[i]==1:
            predseq.append(X_test[j][i])
        else:
            predseq.append(tuple([0, 0, 0, 0]))
        
print(len(predseq))
sequence = one_hot_decode(predseq)
print(sequence)